In [1]:
"""
Shot prompt
4-shot prompt
"""

'\nShot prompt\n4-shot prompt\n'

In [3]:
# define variables
NUM_FILES = 361
FOLDER_NAME = '../../determining_files_rename/random_samples_variables'
GPT_SAVED_FILE_NAME = 'rename_variable_gpt'

In [4]:
# read in files from folder random_cells 
random_cells = []

# read in cells
for i in range(NUM_FILES):
    file_name = f'{FOLDER_NAME}/{i}.py'
    with open(file_name, 'r') as f:
        random_cells.append(f.read())

# read in readmes
with open(f'{FOLDER_NAME}/readmes.txt', 'r') as f:
    readmes = eval(f.read())

In [5]:
task = "Rename the variable specified in the code delimited by triple backticks to a more meaningful name that reflects its usage and/or aligns with the project's purpose. Do not add, remove, or change anything else. Structure your response under the following headings: 'New variable name' (only one new variable name), 'Updated code' (the full code cell with the variable renamed), and 'Explanation' (a 1-2 sentence explanation of the new variable name)."

ex1_input = """Project purpose:
This project is about analyzing top movie genre trends from 2000 to 2010. We will look at the top genres and top movies for each year.

Variable to rename:
'dat'

Code:
```python
import pandas as pd

dat = pd.read_csv('data.csv')
dat.head()
```"""

ex1_output = """New variable name:
'movie_data_df'

Updated code:
```python
import pandas as pd

movie_data_df = pd.read_csv('data.csv')
movie_data_df.head()
```

Explanation:
The new variable name 'movie_data_df' reflects its usage as a dataframe containing movie data."""

ex2_input = """Project purpose:
The focus of this project is determining the most common crime types in LA.

Variable to rename:
'group'

Code:
```python
group = df.groupby('gender')

for gender, grouped_data in group:
    print(f"Gender: {gender}")
    print(grouped_data['crime_type'].value_counts())
    print("\n")
```"""

ex2_output = """New variable name:
'gender_crime_groups'

Updated code:
```python
gender_crime_groups = df.groupby('gender')

for gender, grouped_data in gender_crime_groups:
    print(f"Gender: {gender}")
    print(grouped_data['crime_type'].value_counts())
    print("\n")
```

Explanation:
The new variable name 'gender_crime_groups' aligns with its purpose of grouping crime data by gender."""

ex3_input = """Project purpose:
This repository contains a collection of graph theory assignments for MAT381.

Variable to rename:
'queue'

Code:
```python
queue[0] = queue[0].lower()
while queue:
    vertex = queue.popleft()
    print(vertex, end=' ')

    for neighbor in graph[vertex]:
        if neighbor not in visited:
            queue.append(neighbor)
            visited.add(neighbor)
```"""

ex3_output = """New variable name:
'vertex_queue'

Updated code:
```python
vertex_queue[0] = vertex_queue[0].lower()
while vertex_queue:
    vertex = vertex_queue.popleft()
    print(vertex, end=' ')

    for neighbor in graph[vertex]:
        if neighbor not in visited:
            vertex_queue.append(neighbor)
            visited.add(neighbor)
```

Explanation:
By renaming 'queue' to 'vertex_queue', it's now clearer that the variable is being used to store vertices."""

ex4_input = """Project purpose:
This project is for analyzing the relationship between the number of hours studied and the exam scores of students.

Variable to rename:
'result'

Code:
```python
result, info = pearsonr(queue['Hours'], queue['Scores'])
print(result)
```"""

ex4_output = """New variable name:
'hours_scores_correlation'

Updated code:
```python
hours_scores_correlation, info = pearsonr(queue['Hours'], queue['Scores'])
print(hours_scores_correlation)
```

Explanation:
The variable name 'hours_scores_correlation' reflects the purpose of the variable, which is to store the correlation between the number of hours studied and the exam scores of students."""

In [6]:
# Estimate cost
import sys
sys.path.append("../../..")
import utils

def estimate_tokens():
    in_tok = ''
    out_tok = ''
    for i, cell_src in enumerate(random_cells):
        # estimate prompt
        in_tok += task + ex1_input + ex1_output + ex2_input + ex2_output + ex3_input + ex3_output + ex4_input + ex4_output
        in_tok += f"Project purpose:\n{readmes[i]}\n\nVariable to rename:\n'variable_def'\n\nCode:\n```python\n{cell_src}\n```"
        # estimate response
        out_tok += cell_src
    return in_tok, out_tok

in_tok, out_tok = estimate_tokens()

utils.gpt_35_turbo_token_dollar_cost(in_tok, out_tok)

0.7451805

In [7]:
# # rename using GPT
# import openai
# openai.api_key = my_key

# # GPT
# def rename(purpose, cell_src, name):
#     while True:
#         try:
#             completion = openai.ChatCompletion.create(
#                 model="gpt-3.5-turbo",
#                 temperature=0,
#                 messages = [
#                     {"role" : "user", "content" : task},
#                     {"role" : "user", "content" : ex1_input},
#                     {"role" : "assistant", "content" : ex1_output},
#                     {"role" : "user", "content" : ex2_input},
#                     {"role" : "assistant", "content" : ex2_output},
#                     {"role" : "user", "content" : ex3_input},
#                     {"role" : "assistant", "content" : ex3_output},
#                     {"role" : "user", "content" : ex4_input},
#                     {"role" : "assistant", "content" : ex4_output},
#                     {"role" : "user", "content" : f"Project purpose:\n{purpose}\n\nVariable to rename:\n'{name}'\n\nCode:\n```python\n{cell_src}\n```"}
#                 ]
#             )
#         except Exception as e:
#             if 'maximum context length' in str(e):
#                 print('...Error.. too long...' + str(e))
#                 return 'length', None
#             else:
#                 print('...Error.. trying again...' + str(e))
#         else:
#             break
#     return completion.choices[0].finish_reason, completion.choices[0].message["content"]

# gpt_results = []
# for i, cell_src in enumerate(random_cells):
#     print(f'Processing file {i}')
#     finish_reason, result = rename(readmes[i], cell_src, 'variable_def')
#     print(f'File {i} - {finish_reason}')
#     gpt_results.append({'reason': finish_reason, 'result': result})

# # save the results to a file
# with open(GPT_SAVED_FILE_NAME, 'w') as f:
#     f.write(str(gpt_results))

In [8]:
# read in gpt result from file
with open(GPT_SAVED_FILE_NAME, 'r') as f:
    gpt_results = eval(f.read())

In [9]:
# now split the data into files
gpt_new_names = []
gpt_new_code = []
gpt_explanation = []

for i, result in enumerate(gpt_results):
    if result['reason'] == 'stop':
        # split the result
        first_split = result['result'].split('New variable name:')[1].split('Updated code:')
        updated_name = first_split[0].strip()
        second_split = first_split[1].split('Explanation:')
        updated_code = second_split[0].strip()
        explanation = second_split[1].strip()

        # update name
        if len(updated_name.split('\'')) == 3:
            updated_name = updated_name.split('\'')[1]

        # update name
        if len(updated_name.split('`')) == 3:
            updated_name = updated_name.split('`')[1]

        # get the updated code
        updated_code = updated_code.split('```')[1]
        if updated_code.startswith('python'):
            updated_code = updated_code[6:]
        updated_code = updated_code.strip('\n')
        
        # store
        gpt_new_names.append(updated_name)
        gpt_new_code.append(updated_code)
        gpt_explanation.append(explanation)
    else:
        # if we error we assume nothing
        gpt_new_names.append(None)
        gpt_new_code.append(None)
        gpt_explanation.append(None)

In [10]:
# write gpt new names to a file
with open('gpt_new_names.txt', 'w') as f:
    for name in gpt_new_names:
        f.write(f'{name}\n')

In [11]:
# Count the number of times the variable name is/isn't successfully changed
import sys
sys.path.append('../../determining_files_rename')
from ast_determine_usable_items import compare_code

pass_count = 0
fail_count = 0

failed_ids = []
length_failed = 0
length_passed = 0

for i in range(NUM_FILES):
    print("Processing file", i)
    if gpt_new_names[i] is None or gpt_new_code[i] is None or gpt_explanation[i] is None:
        fail_count += 1
        failed_ids.append(i)
        length_failed += len(random_cells[i])
        print(i, "first fail")
    elif compare_code(random_cells[i], gpt_new_code[i], 'variable_def', gpt_new_names[i]):
        pass_count += 1
        print(i, "pass")
        length_passed += len(random_cells[i])
    else:
        fail_count += 1
        print(i, "second fail")
        length_failed += len(random_cells[i])
        failed_ids.append(i)

print(f'Pass count: {pass_count}, {pass_count / (pass_count + fail_count) * 100}%')
print(f'Fail count: {fail_count}, {fail_count / (pass_count + fail_count) * 100}%')

print(f'Average length of failed files: {length_failed / fail_count}')
print(f'Average length of passed files: {length_passed / pass_count}')

Processing file 0
0 pass
Processing file 1
1 pass
Processing file 2
2 pass
Processing file 3
3 pass
Processing file 4
4 pass
Processing file 5
5 pass
Processing file 6
6 pass
Processing file 7
7 pass
Processing file 8
8 pass
Processing file 9
9 pass
Processing file 10
10 pass
Processing file 11
11 pass
Processing file 12
12 pass
Processing file 13
13 pass
Processing file 14
14 pass
Processing file 15
15 pass
Processing file 16
16 pass
Processing file 17
17 pass
Processing file 18
18 pass
Processing file 19
19 pass
Processing file 20
20 pass
Processing file 21
21 pass
Processing file 22
22 pass
Processing file 23
23 pass
Processing file 24
24 pass
Processing file 25
25 pass
Processing file 26
26 pass
Processing file 27
27 pass
Processing file 28
28 pass
Processing file 29
29 pass
Processing file 30
30 pass
Processing file 31
31 pass
Processing file 32
32 pass
Processing file 33
33 pass
Processing file 34
34 second fail
Processing file 35
35 pass
Processing file 36
36 pass
Processing fil

In [12]:
failed_ids

[34,
 46,
 70,
 100,
 102,
 138,
 140,
 177,
 185,
 190,
 199,
 201,
 208,
 216,
 217,
 235,
 236,
 238,
 262,
 266,
 276,
 284,
 289,
 307,
 309,
 349]

In [13]:
gpt_new_names[46]

'final_layer_output'

In [14]:
print(random_cells[46])

from tensorflow.contrib.layers import flatten

def LeNet(x):
    mu = 0
    sigma = 0.1
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 3, 6), mean=mu, stddev=sigma))
    conv1_b = tf.Variable(tf.zeros(6))
    conv1 = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b
    conv1 = tf.nn.relu(conv1)
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean=mu, stddev=sigma))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2 = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    conv2 = tf.nn.relu(conv2)
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    fc0 = flatten(conv2)
    fc1_W = tf.Variable(tf.truncated_normal(shape=(400, 120), mean=mu, stddev=sigma))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1 = tf.matmul(fc0, fc1_W) + fc1_b
    fc1 = tf.nn.relu(fc1)
    fc2_W = tf

In [15]:
print(gpt_new_code[46])

def model_arc(x):
    mu = 0
    sigma = 0.1
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 3, 9), mean=mu, stddev=sigma))
    conv1_b = tf.Variable(tf.zeros(9))
    conv1 = tf.nn.bias_add(tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID'), conv1_b)
    conv1 = tf.nn.relu(conv1)
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    conv2_W = tf.Variable(tf.truncated_normal(shape=(3, 3, 9, 27), mean=mu, stddev=sigma))
    conv2_b = tf.Variable(tf.zeros(27))
    conv2 = tf.nn.bias_add(tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID'), conv2_b)
    conv2 = tf.nn.relu(conv2)
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    conv3_W = tf.Variable(tf.truncated_normal(shape=(1, 1, 27, 81), mean=mu, stddev=sigma))
    conv3_b = tf.Variable(tf.zeros(81))
    conv3 = tf.nn.bias_add(tf.nn.conv2d(conv2, conv3_W, strides=[1, 1, 1, 1], padding='VALID'), conv3_b)
    co